In [281]:
import sys
import nltk
import sklearn
import pandas
import numpy

import pandas as pd
import numpy as np
df = pd.read_csv('Bug Reports Dataset.csv')
print (df.head(5))

   Bug ID Product Component          Assignee    Status Classification  \
0  528905     JDT      Core    jdt-core-inbox  RESOLVED        Eclipse   
1  530231     JDT      Core    jdt-core-inbox  RESOLVED        Eclipse   
2  530758     JDT      Core          jarthana  VERIFIED        Eclipse   
3  531990     JDT      Core    jdt-core-inbox  VERIFIED        Eclipse   
4  532137     JDT      Core  register.eclipse  VERIFIED        Eclipse   

  Priority            Opened  \
0       P1  12/18/2017 11:46   
1       P1    1/24/2018 5:16   
2       P1    2/5/2018 20:38   
3       P1     3/5/2018 0:49   
4       P1    3/7/2018 16:17   

                                             Summary       Category  
0  JDT UI Gerrit failing with "invalid location f...  Configuration  
1  Compilation failure in M20180123-0400 in jdt.c...  Configuration  
2                    Build failure in I20180205-2000  Configuration  
3                    Build failure in I20180304-2000  Configuration  
4  Todays up

In [282]:
classes = df['Priority']
print(classes.value_counts())

P3    925
P2    539
P1    294
P5    225
P4    155
Name: Priority, dtype: int64


In [283]:
from sklearn.utils import resample
df_minority1 = df[df.Priority=='P1']
df_minority2 = df[df.Priority=='P2']
df_minority3 = df[df.Priority=='P3']
df_minority4 = df[df.Priority=='P4']
df_minority5 = df[df.Priority=='P5']

In [284]:
df_minority_upsampled1 = resample(df_minority1, 
                                 replace=True,     # sample with replacement
                                 n_samples=600)    # to match majority class
df_minority_upsampled2 = resample(df_minority2, 
                                 replace=True,     # sample with replacement
                                 n_samples=400)    # to match majority class
df_minority_upsampled3 = resample(df_minority3, 
                                 replace=True,     # sample with replacement
                                 n_samples=250)    # to match majority class
df_minority_upsampled4 = resample(df_minority4, 
                                 replace=True,     # sample with replacement
                                 n_samples=550)    # to match majority class
df_minority_upsampled5 = resample(df_minority5, 
                                 replace=True,     # sample with replacement
                                 n_samples=400)

In [285]:
df = pd.concat([df, df_minority_upsampled1, df_minority_upsampled2, df_minority_upsampled3, df_minority_upsampled4, df_minority_upsampled5])

In [286]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
Y = encoder.fit_transform (classes)
print(classes[:8])
print(Y[:8])
summary = df['Summary']
print(summary[:10])

0    P1
1    P1
2    P1
3    P1
4    P1
5    P1
6    P1
7    P1
Name: Priority, dtype: object
[0 0 0 0 0 0 0 0]
0    JDT UI Gerrit failing with "invalid location f...
1    Compilation failure in M20180123-0400 in jdt.c...
2                      Build failure in I20180205-2000
3                      Build failure in I20180304-2000
4    Todays update produces compile error but javac...
5    Build 4.9.0 I-Build: I20180615-0300 failed due...
6    Cannot Start Eclipse after Plugin 'Ecore Diagr...
7                             Crash in parseXMLElement
8    org.mozilla.plugincontainer - EXC_BAD_ACCESS -...
9          English language pack not updated with 61.0
Name: Summary, dtype: object


In [287]:
import re
processed = re.sub(r'[\d]', ' ',str(summary))
print(processed)

        JDT UI Gerrit failing with "invalid location f...
        Compilation failure in M        -     in jdt.c...
                          Build failure in I        -    
                          Build failure in I        -    
        Todays update produces compile error but javac...
                              ...                        
        [JFace] ResourceManager throws ConcurrentModif...
        NullPointerException in EcoreReconstructorSwit...
        [Share] Problem sharing when project uses loca...
        NullPointerException in org.eclipse.emf.facet....
        NullPointerException in org.eclipse.emf.facet....
Name: Summary, Length:     , dtype: object


In [288]:
processed = summary.str.replace(r'(\d)' , '')
processed = processed.str.replace(r'^\D?(\d{3})\D?\D?(\d{3})\D?(\d{4})$' , ' ')
processed = processed.str.replace(r'\s+' , ' ')
processed = processed.str.replace(r'[ \t]+$' , ' ')
processed = processed.str.replace(r'[&#();*]' , ' ')
processed = processed.str.replace(r'[,:]' , ' ')
processed = processed.str.replace(r'[.!?\\-]' , ' ')
processed = processed.str.replace(r'[//]' , ' ')
processed = processed.str.replace(r'[|]' , ' ')
processed = processed.str.replace(r'\'' , ' ')
processed = processed.str.lower()
processed = processed.str.replace(r'\s[a-z]\s' , ' ')
processed = processed.str.replace(r'\[[^\]]*\]',' ')
processed = processed.str.replace(r'\"',' ')
processed = processed.str.replace(r'\`',' ')
print(processed)

0       jdt ui gerrit failing with  invalid location f...
1            compilation failure in  in jdt compiler tool
2                                       build failure in 
3                                       build failure in 
4       todays update produces compile error but javac...
                              ...                        
2041      resourcemanager throws concurrentmodificatio...
1911    nullpointerexception in ecorereconstructorswit...
2036      problem sharing when project uses local conn...
1992    nullpointerexception in org eclipse emf facet ...
1994    nullpointerexception in org eclipse emf facet ...
Name: Summary, Length: 4338, dtype: object


In [289]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Muhammad
[nltk_data]     Wasim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [290]:
from nltk.corpus import stopwords

In [291]:
stop_words = set(stopwords.words('english')) 

In [292]:
processed = processed.apply(lambda x: ' ' .join(term for term in x.split() if term not in stop_words))

In [293]:
print(processed)

0       jdt ui gerrit failing invalid location system ...
1                   compilation failure jdt compiler tool
2                                           build failure
3                                           build failure
4       todays update produces compile error javac com...
                              ...                        
2041    resourcemanager throws concurrentmodificatione...
1911    nullpointerexception ecorereconstructorswitchb...
2036    problem sharing project uses local connection ...
1992    nullpointerexception org eclipse emf facet wid...
1994    nullpointerexception org eclipse emf facet efa...
Name: Summary, Length: 4338, dtype: object


In [294]:
#from nltk.stem.snowball import SnowballStemmer

In [295]:
#stemmer = SnowballStemmer("english")

In [296]:
#processed = processed.apply(lambda x: ' ' .join(stemmer.stem(term) for term in x.split()))

In [297]:
#print(processed)

In [298]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\Muhammad
[nltk_data]     Wasim\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [299]:
from nltk.stem import WordNetLemmatizer 

In [300]:
lemmatizer = WordNetLemmatizer() 

In [301]:
processed = processed.apply(lambda x: ' ' .join(lemmatizer.lemmatize(term) for term in x.split()))

In [302]:
processed

0       jdt ui gerrit failing invalid location system ...
1                   compilation failure jdt compiler tool
2                                           build failure
3                                           build failure
4       today update produce compile error javac compi...
                              ...                        
2041    resourcemanager throw concurrentmodificationex...
1911    nullpointerexception ecorereconstructorswitchb...
2036    problem sharing project us local connection me...
1992    nullpointerexception org eclipse emf facet wid...
1994    nullpointerexception org eclipse emf facet efa...
Name: Summary, Length: 4338, dtype: object

In [303]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Muhammad
[nltk_data]     Wasim\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [304]:
from nltk.tokenize import word_tokenize

In [305]:
allwords = []
for message in processed:
    words = word_tokenize(message)
    for w in words:
        allwords.append(w)
        
allwords = nltk.FreqDist(allwords)

print('Number of words: {}'.format(len(allwords)))
print('most common words: {}'.format(allwords.most_common(50)))

Number of words: 3720
most common words: [('test', 800), ('eclipse', 336), ('failure', 286), ('org', 277), ('error', 260), ('build', 244), ('intermittent', 241), ('>', 238), ('memory', 208), ('java', 190), ('theme', 176), ('color', 167), ('add', 164), ('leak', 162), ('crash', 157), ('dark', 152), ('j', 148), ('ui', 146), ('tab', 146), ('update', 143), ('firefox', 139), ('plugin', 136), ('<', 136), ('exception', 134), ('type', 132), ('use', 130), ('assertion', 129), ('internal', 126), ('nullpointerexception', 126), ('code', 122), ('button', 121), ('timed', 121), ('http', 120), ('page', 119), ('method', 119), ('cpu', 118), ('jdt', 114), ('dialog', 113), ('dom', 112), ('src', 109), ('window', 108), ('content', 107), ('new', 105), ('file', 102), ('using', 102), ('null', 101), ('high', 100), ('browser', 95), ('=', 94), ('failing', 93)]


In [306]:
processed[0]

0    jdt ui gerrit failing invalid location system ...
0    jdt ui gerrit failing invalid location system ...
0    jdt ui gerrit failing invalid location system ...
0    jdt ui gerrit failing invalid location system ...
Name: Summary, dtype: object

In [307]:
word_features = allwords.most_common(50)
word_features = [i[0] for i in word_features]
print(word_features)

['test', 'eclipse', 'failure', 'org', 'error', 'build', 'intermittent', '>', 'memory', 'java', 'theme', 'color', 'add', 'leak', 'crash', 'dark', 'j', 'ui', 'tab', 'update', 'firefox', 'plugin', '<', 'exception', 'type', 'use', 'assertion', 'internal', 'nullpointerexception', 'code', 'button', 'timed', 'http', 'page', 'method', 'cpu', 'jdt', 'dialog', 'dom', 'src', 'window', 'content', 'new', 'file', 'using', 'null', 'high', 'browser', '=', 'failing']


In [308]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [309]:
vectorizer = TfidfVectorizer()

In [310]:
X = vectorizer.fit_transform(processed).toarray()

In [311]:
print(X.shape)

(4338, 3670)


In [312]:
pd.DataFrame(X, columns=vectorizer.get_feature_names()) 

,__cfrunloopservicemachport,__format,__nscfdictionary,_cairo_array_allocate,_psutil_osx,_ssl,_tests,aa,aarch,ab,...,youtube,yubikey,zero,zerohedge,zhihu,zlib,zombie,zoom,zoomed,zooming
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4334,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4335,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4336,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [313]:
Y = df['Priority']

In [314]:
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [315]:
print(Y.value_counts())

P3    1175
P2     939
P1     894
P4     705
P5     625
Name: Priority, dtype: int64


In [316]:
from sklearn.ensemble import RandomForestClassifier
text_classifier = RandomForestClassifier(n_estimators=100, random_state=0)  
text_classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [317]:
predictions = text_classifier.predict(X_test)

In [318]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,predictions))  
print(classification_report(y_test,predictions))  
print(accuracy_score(y_test, predictions))

[[185   6   6   1   0]
 [  3 158  32   0   0]
 [ 10  17 194   3   2]
 [  0   0   0 139   0]
 [  0   0   2   1 109]]
              precision    recall  f1-score   support

          P1       0.93      0.93      0.93       198
          P2       0.87      0.82      0.84       193
          P3       0.83      0.86      0.84       226
          P4       0.97      1.00      0.98       139
          P5       0.98      0.97      0.98       112

    accuracy                           0.90       868
   macro avg       0.92      0.92      0.92       868
weighted avg       0.90      0.90      0.90       868

0.9043778801843319


In [319]:
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)

In [320]:
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))

[[175   5   8   5   5]
 [ 15 163  10   1   4]
 [ 20  51 137  14   4]
 [  0   0   0 139   0]
 [  0   0   0   3 109]]
              precision    recall  f1-score   support

          P1       0.83      0.88      0.86       198
          P2       0.74      0.84      0.79       193
          P3       0.88      0.61      0.72       226
          P4       0.86      1.00      0.92       139
          P5       0.89      0.97      0.93       112

    accuracy                           0.83       868
   macro avg       0.84      0.86      0.84       868
weighted avg       0.84      0.83      0.83       868

0.8329493087557603


In [321]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
logreg = LogisticRegression(solver='liblinear', random_state=0)
#fit the model
logreg.fit(X_train, y_train)
y_pred_test = logreg.predict(X_test)

In [322]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,y_pred_test))  
print(classification_report(y_test,y_pred_test))  
print(accuracy_score(y_test, y_pred_test))

[[167  13  18   0   0]
 [ 10 151  26   2   4]
 [ 14  21 181   6   4]
 [  0   2   1 136   0]
 [  1   1   7   6  97]]
              precision    recall  f1-score   support

          P1       0.87      0.84      0.86       198
          P2       0.80      0.78      0.79       193
          P3       0.78      0.80      0.79       226
          P4       0.91      0.98      0.94       139
          P5       0.92      0.87      0.89       112

    accuracy                           0.84       868
   macro avg       0.86      0.85      0.85       868
weighted avg       0.84      0.84      0.84       868

0.8433179723502304


In [323]:
from sklearn.tree import DecisionTreeClassifier 
dtree_model = DecisionTreeClassifier(criterion = 'entropy').fit(X_train, y_train) 
dtree_predictions = dtree_model.predict(X_test) 
print("Accuracy:",metrics.accuracy_score(y_test, dtree_predictions))

Accuracy: 0.8894009216589862


In [324]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test,dtree_predictions))  
print(classification_report(y_test,dtree_predictions))  
print(accuracy_score(y_test, dtree_predictions))

[[183   4  10   1   0]
 [  9 162  22   0   0]
 [ 16  20 179   5   6]
 [  0   0   0 139   0]
 [  0   0   2   1 109]]
              precision    recall  f1-score   support

          P1       0.88      0.92      0.90       198
          P2       0.87      0.84      0.85       193
          P3       0.84      0.79      0.82       226
          P4       0.95      1.00      0.98       139
          P5       0.95      0.97      0.96       112

    accuracy                           0.89       868
   macro avg       0.90      0.91      0.90       868
weighted avg       0.89      0.89      0.89       868

0.8894009216589862
